## Day 49 Lecture 2 Assignment

In this assignment, we will apply mean shift clustering to a dataset containing the results of a survey on financial wellbeing.

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, metrics

This dataset contains the results of a survey on a financial wellbeing conducted by the US Consumer Finance Protection Bureau that were published in October 2017. This dataset has a large number of columns, most of which correspond to specific questions on the survey. These codebook for translating the column names to questions can be found here:

https://s3.amazonaws.com/files.consumerfinance.gov/f/documents/cfpb_nfwbs-puf-codebook.pdf

Load the dataset.

In [0]:
# answer goes here

finance= pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/financial_wellbeing.csv')



In [3]:
finance.head()

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,FWB1_4,FWB1_5,FWB1_6,FWB2_1,FWB2_2,FWB2_3,FWB2_4,FSscore,FS1_1,FS1_2,FS1_3,FS1_4,FS1_5,FS1_6,FS1_7,FS2_1,FS2_2,FS2_3,SUBKNOWL1,ACT1_1,ACT1_2,FINGOALS,PROPPLAN_1,PROPPLAN_2,PROPPLAN_3,PROPPLAN_4,MANAGE1_1,MANAGE1_2,MANAGE1_3,MANAGE1_4,...,SOCSEC2,SOCSEC3,LIFEEXPECT,HHEDUC,KIDS_NoChildren,KIDS_1,KIDS_2,KIDS_3,KIDS_4,EMPLOY,EMPLOY1_1,EMPLOY1_2,EMPLOY1_3,EMPLOY1_4,EMPLOY1_5,EMPLOY1_6,EMPLOY1_7,EMPLOY1_8,EMPLOY1_9,RETIRE,MILITARY,Military_Status,agecat,generation,PPEDUC,PPETHM,PPGENDER,PPHHSIZE,PPINCIMP,PPMARIT,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,3,2,3,2,3,2,4,44,3,3,4,3,3,3,4,4,3,4,5,4,3,1,5,4,4,3,4,4,2,4,...,62,-2,-2,4,-1,0,0,0,0,8,0,0,0,0,0,0,0,1,0,1,0,5,8,1,4,1,1,1,7,3,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,3,3,4,2,2,2,3,43,3,3,3,3,4,3,2,4,3,2,5,4,3,0,3,2,2,1,4,4,1,4,...,-2,66,90,2,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,2,1,1,2,6,3,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,5,3,3,1,4,4,4,4,3,3,3,3,...,-2,68,78,3,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,3,2,1,3,6,3,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,-1,-1,-1,-1,3,3,3,3,4,4,2,4,...,-2,-1,-1,-1,-1,0,0,0,0,99,0,0,0,0,0,0,0,0,1,-2,-1,-1,3,3,2,1,1,1,8,3,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,4,3,3,1,3,3,3,3,3,3,3,3,...,-2,65,75,2,1,0,0,0,0,2,0,1,0,1,0,0,0,0,0,-2,0,5,2,4,2,3,1,5,7,1,1,2,4,0,0,1,0,4,1,4.260668


While the survey questions have the potential for interesting cluster analysis, we will stick to the "score" columns to avoid clustering in an unreasonably high-dimensional space. The columns we are interested in all have "score" in their names; identify and isolate these columns. (There should be 4 in total.)

In [6]:
# answer goes here


X= finance.filter(regex='score')
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6394 entries, 0 to 6393
Data columns (total 4 columns):
FWBscore    6394 non-null int64
FSscore     6394 non-null int64
LMscore     6394 non-null int64
KHscore     6394 non-null float64
dtypes: float64(1), int64(3)
memory usage: 199.9 KB


In [45]:
X.describe()

,FWBscore,FSscore,LMscore,KHscore
count,6394.000000,6394.000000,6394.000000,6394.000000
mean,56.034094,50.719112,2.506256,-0.056935
std,14.154676,12.656921,0.755215,0.814936
min,-4.000000,-1.000000,0.000000,-2.053000
25%,48.000000,42.000000,2.000000,-0.570000
50%,56.000000,50.000000,3.000000,-0.188000
75%,65.000000,57.000000,3.000000,0.712000
max,95.000000,85.000000,3.000000,1.267000


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [0]:
# answer goes here

scaler = StandardScaler()
X_std = scaler.fit_transform(X)



Run mean shift clustering on the scores in the survey dataset using the default bandwidth. Then answer the following by printing or typing as appropriate:

- How many clusters are produced? 
- What are the cluster centers?
- How many responses are assigned to each cluster?
- Are these results reasonable? If not, what changes should we make?

In [39]:
# answer goes here

mean_shift= MeanShift()
mean_shift.fit(X_std)



MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [40]:
mean_shift.get_params

<bound method BaseEstimator.get_params of MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)>

In [43]:
print(pd.DataFrame(scaler.inverse_transform(mean_shift.cluster_centers_), columns=X.columns))

    FWBscore    FSscore   LMscore   KHscore
0  57.437953  50.121439  2.814341  0.217414
1  -1.000000  -1.000000  0.250000 -1.826000


In [41]:
mean_clusters= pd.DataFrame(mean_shift.cluster_centers_, columns= X.columns)
mean_clusters

,FWBscore,FSscore,LMscore,KHscore
0,0.099188,-0.047225,0.407975,0.336678
1,-4.029662,-4.086551,-2.987800,-2.170973


In [62]:
print('The number of cluster produced is:', len(mean_shift.cluster_centers_))
print('The cluster scaled centers are \n{}'.format(mean_clusters))
print('The cluster unscaled centers are \n{}'.format(pd.DataFrame(scaler.inverse_transform(mean_shift.cluster_centers_), columns=X.columns)))
print('The number of responses assigned to each cluster are: \n{}'.format(pd.Series(mean_shift.labels_).value_counts()))
print('The results are not reasonable. A possible solution is to adjust the bandwith.')

The number of cluster produced is: 2
The cluster scaled centers are 
   FWBscore   FSscore   LMscore   KHscore
0  0.099188 -0.047225  0.407975  0.336678
1 -4.029662 -4.086551 -2.987800 -2.170973
The cluster unscaled centers are 
    FWBscore    FSscore   LMscore   KHscore
0  57.437953  50.121439  2.814341  0.217414
1  -1.000000  -1.000000  0.250000 -1.826000
The number of responses assigned to each cluster are: 
0    6326
1      68
dtype: int64
The results are not reasonable. A possible solution is to adjust the bandwith.


In [66]:
# the silhouette score is 
labels = MeanShift().fit_predict(X_std)
print(metrics.silhouette_score(X_std, labels, metric='euclidean'))

0.4061338865387727


Try changing the appropriate parameters of the mean shift algorithm to achieve a better clustering result. Answer all of the same questions from the previous clustering step.

In [92]:
# answer goes here

mean_shift_adj= MeanShift(bandwidth=1.6)
mean_shift_adj.fit(X_std)



MeanShift(bandwidth=1.6, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [93]:
mean_clusters_adj= pd.DataFrame(mean_shift_adj.cluster_centers_, columns= X.columns)
mean_clusters_adj

,FWBscore,FSscore,LMscore,KHscore
0,0.174503,-0.021170,0.534077,0.502707
1,-0.319867,-0.396715,-0.983611,-0.814066
2,-0.042782,-3.454437,-3.129681,-2.250390
3,-4.029662,-4.086551,-2.987800,-2.170973
4,2.753080,2.708682,-3.318857,-2.449544
5,1.693276,-3.217394,0.653830,0.943627


In [94]:
print('The number of cluster produced is:', len(mean_shift_adj.cluster_centers_))
print('The cluster scaled centers are \n{}'.format(mean_clusters_adj))
print('The cluster unscaled centers are \n{}'.format(pd.DataFrame(scaler.inverse_transform(mean_shift_adj.cluster_centers_), columns=X.columns)))
print('The number of responses assigned to each cluster are: \n{}'.format(pd.Series(mean_shift_adj.labels_).value_counts()))

The number of cluster produced is: 6
The cluster scaled centers are 
   FWBscore   FSscore   LMscore   KHscore
0  0.174503 -0.021170  0.534077  0.502707
1 -0.319867 -0.396715 -0.983611 -0.814066
2 -0.042782 -3.454437 -3.129681 -2.250390
3 -4.029662 -4.086551 -2.987800 -2.170973
4  2.753080  2.708682 -3.318857 -2.449544
5  1.693276 -3.217394  0.653830  0.943627
The cluster unscaled centers are 
    FWBscore    FSscore   LMscore   KHscore
0  58.503932  50.451180  2.909567  0.352707
1  51.506838  45.698311  1.763475 -0.720294
2  55.428571   7.000000  0.142857 -1.890714
3  -1.000000  -1.000000  0.250000 -1.826000
4  95.000000  85.000000  0.000000 -2.053000
5  80.000000  10.000000  3.000000  0.712000
The number of responses assigned to each cluster are: 
0    4154
1    2137
4      48
2      28
5      16
3      11
dtype: int64


In [95]:
# the silhouette score is 
labels = MeanShift(bandwidth=1.6).fit_predict(X_std)
print(metrics.silhouette_score(X_std, labels, metric='euclidean'))

0.27300608551818184


Adjusted model round 2

In [84]:
mean_shift_adj= MeanShift(bandwidth=1.8)
mean_shift_adj.fit(X_std)


MeanShift(bandwidth=1.8, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [85]:
mean_clusters_adj= pd.DataFrame(mean_shift_adj.cluster_centers_, columns= X.columns)
mean_clusters_adj

,FWBscore,FSscore,LMscore,KHscore
0,0.128907,-0.052100,0.459315,0.412017
1,-0.128015,-3.112041,-3.171720,-2.201925
2,-4.029662,-4.086551,-2.987800,-2.170973


In [86]:
print('The number of cluster produced is:', len(mean_shift_adj.cluster_centers_))
print('The cluster scaled centers are \n{}'.format(mean_clusters_adj))
print('The cluster unscaled centers are \n{}'.format(pd.DataFrame(scaler.inverse_transform(mean_shift_adj.cluster_centers_), columns=X.columns)))
print('The number of responses assigned to each cluster are: \n{}'.format(pd.Series(mean_shift_adj.labels_).value_counts()))

The number of cluster produced is: 3
The cluster scaled centers are 
   FWBscore   FSscore   LMscore   KHscore
0  0.128907 -0.052100  0.459315  0.412017
1 -0.128015 -3.112041 -3.171720 -2.201925
2 -4.029662 -4.086551 -2.987800 -2.170973
The cluster unscaled centers are 
    FWBscore    FSscore   LMscore   KHscore
0  57.858591  50.059742  2.853110  0.278806
1  54.222222  11.333333  0.111111 -1.851222
2  -1.000000  -1.000000  0.250000 -1.826000
The number of responses assigned to each cluster are: 
0    5972
1     405
2      17
dtype: int64


In [87]:
# the silhouette score is 
labels = MeanShift(bandwidth=1.8).fit_predict(X_std)
print(metrics.silhouette_score(X_std, labels, metric='euclidean'))

0.329682272456272
